# Hackhaton 02 - Sistemas de Recomendação 

Dicionário dos datasets: <br>

#### DNC_orders_dataset 
order_id – Id do pedido de compra; <br>
customer_id – Id do consumidor que realizou a compra; <br>
order_status – Status da compra; <br>
<br>
#### DNC_order_items_dataset
order_id – Id do pedido de compra; <br>
order_item_id – Número de produtos em uma mesma compra; <br>
product_id – Id do produto; <br>
price - preço; <br>
<br>

#### DNC_items_reviews

review_id - Id do review; <br>
order_id - Id do pedido; <br>
review_score - Nota do pedido; <br>
<br>

#### DNC_products_dataset

product_id – Id do produto; <br>
product_category_name – Categoria do produto; <br>
product_name_length – Tamanho do nome do produto; <br>
product_description_length – comprimento do produto; <br>
products_weight_g – peso do produto em g; <br>
products_lenght_cm – comprimento do produto em cm; <br>
products_height_cm – Altura do produto em cm; <br>
product_widht_cm – largura do produto em cm; <br>

autor: Gianlucca Vick

## Importação das bibliotecas necessárias

In [140]:
import pandas as pd 
import os 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

## Carregamento dos 4 conjuntos de dados que serão utilizados para criar o sistema de recomendação, depois removemos as colunas indesejadas para nosso sistema de recomendação (Unnamed: 0), pois não agrega nenhum valor

In [120]:
df_orders = pd.read_csv('DNC_orders_dataset.csv')
df_orders_items = pd.read_csv('DNC_order_items_dataset.csv')
df_orders_reviews = pd.read_csv('DNC_order_reviews_dataset.csv')
df_products = pd.read_csv('DNC_products_dataset.csv')

df_orders.drop(columns=['Unnamed: 0'], inplace=True)
df_orders_items.drop(columns=['Unnamed: 0'], inplace=True)
df_orders_reviews.drop(columns=['Unnamed: 0'], inplace=True)
df_products.drop(columns=['Unnamed: 0'], inplace=True)

display(df_orders), display(df_orders_items), display(df_orders_reviews), display(df_products)

,order_id,customer_id,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered
...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered


,order_id,order_item_id,product_id,price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90
...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,299.99
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,350.00
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,99.90
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,55.99


,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5
...,...,...,...
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,700.0,31.0,13.0,20.0


(None, None, None, None)

## Analisando os dados

In [121]:
df_products['product_id'].nunique() #Todos os valores de produtos são únicos
m['customer_id'].nunique(), m.shape  #Nem todos os valores de clientes são únicos (existem clientes repetidos)
m['order_id'].nunique(), m.shape #Nem todos os valores de pedidos são únicos (também existem pedidos repetidos)
df_orders_items['product_id'].nunique(), df_orders_items.shape #Existem muitos valores repetidos de produtos 
df_orders_items['order_id'].nunique(), df_orders_items.shape  #Também existem valores repetidos de pedidos

(98666, (112650, 4))

## Merge da coluna reviews para a coluna orders, para criarmos um dataframe com todas as informações necessárias, e então criarmos um sistema de recomendação com base nos clientes

In [122]:
m = pd.merge(df_orders, df_orders_reviews, how = 'inner', on = 'order_id')
m

,order_id,customer_id,order_status,review_id,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,a54f0611adc9ed256b57ede6b6eb5114,4
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,8d5266042046a06655c8db133d120ba5,4
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,e73b67b67587f7644d5bd1a52deb1b01,5
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,359d03e676b3c069f62cadba8dd3f6e8,5
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,e50934924e227544ba8246aeb3770dd4,5
...,...,...,...,...,...
99219,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,e262b3f92d1ce917aa412a9406cf61a6,5
99220,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,29bb71b2760d0f876dfa178a76bc4734,4
99221,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,371579771219f6db2d830d50805977bb,5
99222,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,8ab6855b9fe9b812cd03a480a25058a1,2


## Merge das colunas de pedidos e produtos, para saber os pedidos mais realizados e criar um sistema de recomendação com base na similaridade dos produtos

In [123]:
pedidos = pd.merge(m, df_orders_items, how='inner', on='order_id')
df_produtos_final = pd.merge(pedidos, df_products, how='left', on='product_id')
df_produtos_final

,order_id,customer_id,order_status,review_id,review_score,order_item_id,product_id,price,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,a54f0611adc9ed256b57ede6b6eb5114,4,1,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,40.0,268.0,500.0,19.0,8.0,13.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,8d5266042046a06655c8db133d120ba5,4,1,595fac2a385ac33a80bd5114aec74eb8,118.70,perfumaria,29.0,178.0,400.0,19.0,13.0,19.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,e73b67b67587f7644d5bd1a52deb1b01,5,1,aa4383b373c6aca5d8797843e5594415,159.90,automotivo,46.0,232.0,420.0,24.0,19.0,21.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,359d03e676b3c069f62cadba8dd3f6e8,5,1,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,pet_shop,59.0,468.0,450.0,30.0,10.0,20.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,e50934924e227544ba8246aeb3770dd4,5,1,65266b2da20d04dbe00c5c2d3bb7859e,19.90,papelaria,38.0,316.0,250.0,51.0,15.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112367,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,29bb71b2760d0f876dfa178a76bc4734,4,1,f1d4ce8c6dd66c47bbaa8c6781c2a923,174.90,bebes,52.0,828.0,4950.0,40.0,10.0,40.0
112368,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,371579771219f6db2d830d50805977bb,5,1,b80910977a37536adeddd63663f916ad,205.99,eletrodomesticos_2,51.0,500.0,13300.0,32.0,90.0,22.0
112369,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,8ab6855b9fe9b812cd03a480a25058a1,2,1,d1c427060a0f73f6b889a5c7c61f2ac4,179.99,informatica_acessorios,59.0,1893.0,6550.0,20.0,20.0,20.0
112370,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,8ab6855b9fe9b812cd03a480a25058a1,2,2,d1c427060a0f73f6b889a5c7c61f2ac4,179.99,informatica_acessorios,59.0,1893.0,6550.0,20.0,20.0,20.0


## Verificação do tamanho das colunas categóricas para ver se é possível transformar elas em numéricas

In [16]:
print(df_orders['order_status'].value_counts())
print()
print(df_orders_items['order_item_id'].value_counts())
print()
print(df_products['product_category_name']).value_counts()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

1     98666
2      9803
3      2287
4       965
5       460
6       256
7        58
8        36
9        28
10       25
11       17
12       13
13        8
14        7
15        5
16        3
17        3
18        3
19        3
20        3
21        1
Name: order_item_id, dtype: int64

0                               perfumaria
1                                    artes
2                            esporte_lazer
3                                    bebes
4                    utilidades_domesticas
                       ...                
32946                     moveis_decoracao
32947    construcao_ferramentas_iluminacao
32948                      cama_mesa_banho
32949               informatica_acessorios
32950                      cama_mesa_banho
Name: product_category_name, Length: 3

AttributeError: 'NoneType' object has no attribute 'value_counts'

## Verificação dos tipos de dados presentes no conjunto de dados

In [18]:
print(df_orders.dtypes)
print()
print(df_orders_items.dtypes)
print()
print(df_orders_reviews.dtypes)
print()
print(df_products.dtypes)
print()

order_id        object
customer_id     object
order_status    object
dtype: object

order_id          object
order_item_id      int64
product_id        object
price            float64
dtype: object

review_id       object
order_id        object
review_score     int64
dtype: object

product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object



## Verificação da existencia de dados nulos que existem nos datasets, e tratamento destes dados

O único dataset que possui valores nulos é o df_products, onde existem 610 valores nulos para os nomes e a descrição de produtos, e apenas dois valores nulos para o peso, tamanho e altura desses produtos.

In [137]:
df_produtos_final.isna().sum()
df_produtos_final['product_category_name'].isna().sum() #1598

1598

In [24]:
df_produtos_final.fillna(0, inplace=True)
df_produtos_final.isna().sum()

product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
dtype: int64

## Divisão do dataset em 80/20 em treino e teste

In [159]:
df_teste, df_treino = train_test_split(df_produtos_final, test_size=0.80, random_state=42)
print(f'O tamanho da nossa matriz de treinamento ficou de: {df_treino.shape}')
print(f'O tamanho da nossa matriz de teste ficou de: {df_teste.shape}')

O tamanho da nossa matriz de treinamento ficou de: (89898, 15)
O tamanho da nossa matriz de teste ficou de: (22474, 15)


## Testando recomendações simples, como os itens com maiores avaliações

In [162]:
reviews_sem_zeros = df_produtos_final[df_produtos_final['review_score'] != 0]
soma_reviews = pd.DataFrame(reviews_sem_zeros.groupby(['product_id'])['review_score'].sum())
top10 = soma_reviews.sort_values('review_score', ascending=False).head(10)
top10

,review_score
product_id,
aca2eb7d00ea1a7b8ebd4e68314663af,2106
422879e10f46682990de24d770e7f83d,1918
99a4788cb24856965c36a24e339b6058,1879
389d119b48cf3043d311335e499d9c6b,1610
368c6c730842d78016ad823897a372db,1522
53759a2ecddad2bb87a079a1f1519f73,1443
d1c427060a0f73f6b889a5c7c61f2ac4,1426
53b36df67ebb7c41585e8d54d6772e08,1341
154e7e31ebfa092203795c972e5804a6,1260


## Desenvolvimento do sistema de recomendação com base na similaridade dos produtos, caso o produto não esteja na lista, ele recomenda os 10 produtos com maiores reviews

In [157]:
def start_pipeline(dataf):
    return dataf.copy()

def get_item_counts(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'customer_id': pd.Series.nunique}) \
    .rename(columns={'customer_id': 'customer_id_count'}) \
    .set_index('product_id') \
    .sort_values('customer_id_count', ascending = False)

def get_avg_ratings(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'review_score': np.mean}) \
    .rename(columns={'review_score': 'review_score_avg'}) \
    .set_index('product_id') \
    .sort_values('review_score_avg', ascending = False)

def get_n_top_values(dataf, n):
  return dataf.head(n) \
    .index \
    .tolist()

def recomendar(id_item):
  print(f"Item de interesse: {id_item}")
  if id_item in matriz_de_relacionamento_top_item_ratings.index.tolist():
    item_idx = matriz_de_relacionamento_top_item_ratings.index.tolist().index(id_item)
    closest_10_items = np.argsort(-cos_items[item_idx])[1:11]
    print(f"Retornando 10 itens mais próximos de: {id_item}.")
    print()
    return matriz_de_relacionamento_top_item_ratings.index[closest_10_items].tolist()
  else:
    print(f"{id_item}, não está incluso na matriz de recomendação. Retornando os 10 itens mais bem avaliados")
    print()
    return top_item_sellers

top_item_ratings = df_produtos_final \
  .pipe(start_pipeline) \
  .pipe(get_avg_ratings) \
  .pipe(get_n_top_values, n=1500)

top_item_sellers = df_produtos_final \
  .pipe(start_pipeline) \
  .pipe(get_item_counts) \
  .pipe(get_n_top_values, n=10)

print(f'Os dez itens com melhores avaliações são: {top_item_ratings[:10]}')
print()
print(f'Os dez itens com maiores vendas são: {top_item_sellers}')

print(pedidos.pipe(start_pipeline).pipe(get_avg_ratings))
print()

matriz_de_relacionamento_top_item_ratings = pedidos[pedidos["product_id"].isin(top_item_ratings)] \
  .pivot_table(index="product_id", columns="customer_id", values="review_score").fillna(0)

print(f'O tamanho da matriz de relacionamento dos produtos é de: {matriz_de_relacionamento_top_item_ratings.shape}')
print()

cos_items = cosine_similarity(matriz_de_relacionamento_top_item_ratings)

print(cos_items, cos_items.shape)
print()

id_item_interesse = "00066f42aeeb9f3007548bb9d3f33c38" #Produto com maior número de review
ids_10_mais_proximos = recomendar(id_item_interesse)
print(ids_10_mais_proximos)
print()

#Validação das categorias dos produtos recomendados 

lista_categoria = []

categoria = df_produtos_final.loc[df_produtos_final['product_id'] == id_item_interesse]['product_category_name']

for item in ids_10_mais_proximos:
    categoria = df_produtos_final.loc[df_produtos_final['product_id'] == item]['product_category_name']
    lista_categoria.append(categoria)

print(f'Nosso produto tem a categoria de: {categoria}')
print()

print(lista_categoria)


Os dez itens com melhores avaliações são: ['00066f42aeeb9f3007548bb9d3f33c38', '60314cbf527af04ebc935be7c02dc1cd', '5fe23e6307c231e287f13c0a4be06c88', '5fe490e61e7a37f2f0d1ad1771ac027d', 'c696cd2db7870be08c37822f7c0ef831', '5fef486f2057e10fd91e167348812b7a', '5ff4076c0f01eeba4f728c9e3fa2653c', '5ff59c5f2db7600fa8143442c8b1e4f6', 'c68b419d9c6038271b85bac98adb0fc9', '5ffa5bcfb017e836862c86ec3fbf9045']

Os dez itens com maiores vendas são: ['99a4788cb24856965c36a24e339b6058', 'aca2eb7d00ea1a7b8ebd4e68314663af', '422879e10f46682990de24d770e7f83d', 'd1c427060a0f73f6b889a5c7c61f2ac4', '389d119b48cf3043d311335e499d9c6b', '53b36df67ebb7c41585e8d54d6772e08', '368c6c730842d78016ad823897a372db', '53759a2ecddad2bb87a079a1f1519f73', '154e7e31ebfa092203795c972e5804a6', '2b4609f8948be18874494203496bc318']
                                  review_score_avg
product_id                                        
00066f42aeeb9f3007548bb9d3f33c38               5.0
60314cbf527af04ebc935be7c02dc1cd             

## Testando nosso recomendador com um valor inexistente para ver se ele responde com os dez melhores reviews (já foi incrementado na própria função de recomendação)

In [126]:
ids_10_mais_proximos = recomendar('esse_nao_existe')
ids_10_mais_proximos

Item de interesse: esse_nao_existe
esse_nao_existe, não está incluso na matriz de recomendação. Retornando os 10 itens mais bem avaliados


['99a4788cb24856965c36a24e339b6058',
 'aca2eb7d00ea1a7b8ebd4e68314663af',
 '422879e10f46682990de24d770e7f83d',
 'd1c427060a0f73f6b889a5c7c61f2ac4',
 '389d119b48cf3043d311335e499d9c6b',
 '53b36df67ebb7c41585e8d54d6772e08',
 '368c6c730842d78016ad823897a372db',
 '53759a2ecddad2bb87a079a1f1519f73',
 '154e7e31ebfa092203795c972e5804a6',
 '2b4609f8948be18874494203496bc318']

In [139]:
#00066f42aeeb9f3007548bb9d3f33c38  produto que eu coloquei no recomendador, categoria: perfumaria

lista_recomendados = ['be70253973dbb353054c814633a5d4e4',
 'be6ae1b41a4ba652f1947e0fae960d44',
 'be68ca1484d5bcc92f0949413e5eb445',
 'be59f0b9bd1d08d096188f98168fc987',
 'be5679ce5a2343c40455223d7bef6dfb',
 'be55ae76be8824525f40695d49a5a7b5',
 'be4fa7795030e85bb9fe8285405515fa',
 'be4e205f4a7283ce05c3b9365d283adc',
 'be4daa163f6e100a73d7cddcf2a4c4de',
 'be4cf22bb84e3fd215db5076e3047d51']

lista_categoria = []

categoria = df_produtos_final.loc[df_produtos_final['product_id'] == '00066f42aeeb9f3007548bb9d3f33c38']['product_category_name']
categoria

for item in lista_recomendados:
    categoria = df_produtos_final.loc[df_produtos_final['product_id'] == item]['product_category_name']
    lista_categoria.append(categoria)

print(lista_categoria)
    

[32267    relogios_presentes
Name: product_category_name, dtype: object, 79407    automotivo
Name: product_category_name, dtype: object, 85625    ferramentas_jardim
85626    ferramentas_jardim
Name: product_category_name, dtype: object, 10672    NaN
51762    NaN
78424    NaN
Name: product_category_name, dtype: object, 30008    fashion_bolsas_e_acessorios
Name: product_category_name, dtype: object, 9532     agro_industria_e_comercio
12069    agro_industria_e_comercio
19190    agro_industria_e_comercio
Name: product_category_name, dtype: object, 23301    automotivo
Name: product_category_name, dtype: object, 60476     esporte_lazer
106246    esporte_lazer
Name: product_category_name, dtype: object, 57126    fashion_bolsas_e_acessorios
Name: product_category_name, dtype: object, 50774    esporte_lazer
Name: product_category_name, dtype: object]


## Testando o nosso sistema de recomendação em um dataset de treino para verificar as diferenças contra um dataset de teste

In [150]:
def start_pipeline(dataf):
    return dataf.copy()

def get_item_counts(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'customer_id': pd.Series.nunique}) \
    .rename(columns={'customer_id': 'customer_id_count'}) \
    .set_index('product_id') \
    .sort_values('customer_id_count', ascending = False)

def get_avg_ratings(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'review_score': np.mean}) \
    .rename(columns={'review_score': 'review_score_avg'}) \
    .set_index('product_id') \
    .sort_values('review_score_avg', ascending = False)

def get_n_top_values(dataf, n):
  return dataf.head(n) \
    .index \
    .tolist()

def recomendar(id_item):
  print(f"Item de interesse: {id_item}")
  if id_item in matriz_de_relacionamento_top_item_ratings.index.tolist():
    item_idx = matriz_de_relacionamento_top_item_ratings.index.tolist().index(id_item)
    closest_10_items = np.argsort(-cos_items[item_idx])[1:11]
    print(f"Retornando 10 itens mais próximos de: {id_item}.")
    return matriz_de_relacionamento_top_item_ratings.index[closest_10_items].tolist()
  else:
    print(f"{id_item}, não está incluso na matriz de recomendação. Retornando os 10 itens mais bem avaliados")
    return top_item_sellers

top_item_ratings = df_treino \
  .pipe(start_pipeline) \
  .pipe(get_avg_ratings) \
  .pipe(get_n_top_values, n=1500)

top_item_sellers = df_treino \
  .pipe(start_pipeline) \
  .pipe(get_item_counts) \
  .pipe(get_n_top_values, n=10)

print(f'Os dez itens com melhores avaliações são: {top_item_ratings[:10]}')
print()
print(f'Os dez itens com maiores vendas são: {top_item_sellers}')

print(pedidos.pipe(start_pipeline).pipe(get_avg_ratings))

matriz_de_relacionamento_top_item_ratings = df_treino[df_treino["product_id"].isin(top_item_ratings)] \
  .pivot_table(index="product_id", columns="customer_id", values="review_score").fillna(0)

print(f'O tamanho da matriz de relacionamento dos produtos é de: {matriz_de_relacionamento_top_item_ratings.shape}')
print()

cos_items = cosine_similarity(matriz_de_relacionamento_top_item_ratings)

print(cos_items, cos_items.shape)
print()

id_item_interesse = "00066f42aeeb9f3007548bb9d3f33c38" #Produto com maior número de review
ids_10_mais_proximos = recomendar(id_item_interesse)
print(ids_10_mais_proximos)
print()

#Validação das categorias dos produtos recomendados (por enquanto, é apenas um print das categorias para inspeção visual)
lista_categoria = []

categoria = df_produtos_final.loc[df_produtos_final['product_id'] == id_item_interesse]['product_category_name']

for item in ids_10_mais_proximos:
    categoria = df_produtos_final.loc[df_produtos_final['product_id'] == item]['product_category_name']
    lista_categoria.append(categoria)

print(lista_categoria)

Os dez itens com melhores avaliações são: ['000b8f95fcb9e0096488278317764d19', '8e42c31b4ca1afbb6bbe17efdd666cf5', '8cc0b6265dda0e4e8aced3d1446b80bc', '8cb406882e34a39390cdf5f7f2632589', '8ca6ebfd6293a5b03f104d27c4945718', '8c832a513b94b1ded5579b054615dc20', '8c81b8075a4b824afebae3dc276889eb', '8c6d562d9dde3d9658cf7496f61c1bc0', '8c501fb8530d486fda52b3aa2fd2dca8', '8c38d21062ec6ee492c3500d01150b7a']

Os dez itens com maiores vendas são: ['aca2eb7d00ea1a7b8ebd4e68314663af', '422879e10f46682990de24d770e7f83d', '389d119b48cf3043d311335e499d9c6b', '99a4788cb24856965c36a24e339b6058', '368c6c730842d78016ad823897a372db', '3dd2a17168ec895c781a9191c1e95ad7', 'd1c427060a0f73f6b889a5c7c61f2ac4', '53759a2ecddad2bb87a079a1f1519f73', '2b4609f8948be18874494203496bc318', '154e7e31ebfa092203795c972e5804a6']
                                  review_score_avg
product_id                                        
00066f42aeeb9f3007548bb9d3f33c38               5.0
60314cbf527af04ebc935be7c02dc1cd             

## Observando nosso sistema de recomendação construído, podemos verificar que a maioria das categorias de produtos são semelhantes, porém ainda existem recomendações de categorias bem distintas, tal como a recomendação de um produto automotivo para alguém que acabou de realizar uma compra em um produto de perfumaria.

## Também seria necessário investigar mais a fundo a presença de valores nulos, e qual tratamento seria o melhor para substituir estes.